In [1]:
import os
import pandas as pd
import transformers
import torch
import numpy as np
from Code.utinity import *
from Code.models import *

from tqdm import tqdm

In [2]:
class LLM_u(LLM):


    def get_attention_matrix(self,outputs):
        A = 0
        for layer in range(len(outputs.attentions)):
            X = outputs.attentions[layer][0].to(torch.float32).cpu().detach().numpy()
            for i in range(len(X)):
                Ai = (X[i] + X[i].T)/2
                # Ai = (X[i] + X[i].T)/2 + abs(X[i] - X[i].T)/2
                A = (A+Ai)/2 + abs(A-Ai)/2 # max
        # plt.matshow(A, cmap=plt.cm.Reds);
        A = np.triu(A) # **** 修改
        return A

## 算法设定

以 Llama2 为基础模型，比较现有算法（baseline）和我们设计的算法（ours）

In [4]:
llm = LLM_u('falcon',load_model=True)
# llm = LLM_u('gpt2',load_model=True)
llm.model_name = 'falconuu_test_echo'
solver_baseline = Solver_Baseline(llm) # baseline 
solver_ours_vv = Solver_Echo(llm,repeat=2)
solver_ours_vvv = Solver_Echo(llm,repeat=3)
solver_ours_vvvv = Solver_Echo(llm,repeat=4)
solver_ours_vvvvv = Solver_Echo(llm, repeat=5)
# solver_ours_vvvv = Solver_Ours(llm,repeat=4,attention=True)
solvers = [solver_baseline,solver_ours_vv,solver_ours_vvv,solver_ours_vvvv,solver_ours_vvvvv]

The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.
You are using a model of type RefinedWebModel to instantiate a model of type falcon. This is not supported for all configurations of models and can yield errors.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

falcon 调用成功!


## 算法验证

In [5]:
results = {}
accuracy = {}
# def RunEval(problem,use_temp=True):
def RunEval(problem,use_temp=True):
    results[problem.name] = []
    accuracy[problem.name] = []
    for i,solver in enumerate(solvers):
        # if i == 3:
        #     use_temp = False
        evaluation = Evaluation(problem,solver,later=False,use_temp=use_temp)
        df1,df2 = evaluation.save(filename='result/'+problem.name+'_'+solver.name+'.xlsx')
        results[problem.name].append(df1)
        accuracy[problem.name].append(df2)
    # accuracy[problem.name] = pd.concat([df2,df4],axis=1).T # 添加结果
    return pd.concat(accuracy[problem.name],axis=1).T

### 数据集 1: SLPWC

In [6]:
RunEval(problem=Problem("Data/SLPWC_v1.csv",name='SLPWC'))

读取文件： Temp/temp_embeddings_SLPWC_falconuu_test_echo_baseline.dat
读取文件： Temp/temp_embeddings_SLPWC_falconuu_test_echo_echovv.dat
读取文件： Temp/temp_embeddings_SLPWC_falconuu_test_echo_echovvv.dat
利用 falconuu_test_echo_echovvvv 计算目标词的向量嵌入：


  0%|          | 0/300 [00:00<?, ?it/s]

The current implementation of Falcon calls `torch.scaled_dot_product_attention` directly, this will be deprecated in the future in favor of the `BetterTransformer` API. Please install the latest optimum library with `pip install -U optimum` and call `model.to_bettertransformer()` to benefit from `torch.scaled_dot_product_attention` and future performance optimizations.


保存文件： Temp/temp_embeddings_SLPWC_falconuu_test_echo_echovvvv.dat
利用 falconuu_test_echo_echovvvvv 计算目标词的向量嵌入：


  0%|          | 0/300 [00:00<?, ?it/s]

保存文件： Temp/temp_embeddings_SLPWC_falconuu_test_echo_echovvvvv.dat


,distance_shift,cos_similarity_shift
falconuu_test_echo_baseline,0.583333,0.590000
falconuu_test_echo_echovv,0.540000,0.546667
falconuu_test_echo_echovvv,0.500000,0.496667
falconuu_test_echo_echovvvv,0.463333,0.466667
falconuu_test_echo_echovvvvv,0.483333,0.490000


## 数据集 2: WSD

In [7]:
RunEval(problem=Problem('Data/WSD_v1.csv',name='WSD'))

读取文件： Temp/temp_embeddings_WSD_falconuu_test_echo_baseline.dat
读取文件： Temp/temp_embeddings_WSD_falconuu_test_echo_echovv.dat
读取文件： Temp/temp_embeddings_WSD_falconuu_test_echo_echovvv.dat
利用 falconuu_test_echo_echovvvv 计算目标词的向量嵌入：


  0%|          | 0/862 [00:00<?, ?it/s]

没有找到词: 名称 "君子疾没世而名不称焉。"君子疾没世而名不称焉。"君子疾没世而名不称焉。"君子疾没世而名不称焉。
没有找到词: 那里 这儿瞧瞧,那儿看看,觉得什么都是新鲜的这儿瞧瞧,那儿看看,觉得什么都是新鲜的这儿瞧瞧,那儿看看,觉得什么都是新鲜的这儿瞧瞧,那儿看看,觉得什么都是新鲜的
没有找到词: 精简 精选人物精选人物精选人物精选人物
没有找到词: 摄影 骋足则能追风摄景骋足则能追风摄景骋足则能追风摄景骋足则能追风摄景
保存文件： Temp/temp_embeddings_WSD_falconuu_test_echo_echovvvv.dat
利用 falconuu_test_echo_echovvvvv 计算目标词的向量嵌入：


  0%|          | 0/862 [00:00<?, ?it/s]

没有找到词: 名称 "君子疾没世而名不称焉。"君子疾没世而名不称焉。"君子疾没世而名不称焉。"君子疾没世而名不称焉。"君子疾没世而名不称焉。
没有找到词: 那里 这儿瞧瞧,那儿看看,觉得什么都是新鲜的这儿瞧瞧,那儿看看,觉得什么都是新鲜的这儿瞧瞧,那儿看看,觉得什么都是新鲜的这儿瞧瞧,那儿看看,觉得什么都是新鲜的这儿瞧瞧,那儿看看,觉得什么都是新鲜的
没有找到词: 精简 精选人物精选人物精选人物精选人物精选人物
没有找到词: 摄影 骋足则能追风摄景骋足则能追风摄景骋足则能追风摄景骋足则能追风摄景骋足则能追风摄景
保存文件： Temp/temp_embeddings_WSD_falconuu_test_echo_echovvvvv.dat


,distance_shift,cos_similarity_shift
falconuu_test_echo_baseline,0.446636,0.444316
falconuu_test_echo_echovv,0.334107,0.339907
falconuu_test_echo_echovvv,0.315545,0.328306
falconuu_test_echo_echovvvv,0.323666,0.321346
falconuu_test_echo_echovvvvv,0.324826,0.319026
